# Pump it Up: Data Mining Water Table

## Постановка задачи

В настоящее время тема машинного обучения находится на пике своей популярности. Платформа Drivendata не только организовывает контесты по машинному обучению, но и сотрудничает с социальными и политическими организациями по всему миру, применяя результаты победителей контестов к реальным проблемам. Одна из таких проблем - плохое водоснабжение в стране Танзания в Восточной Африке. В Танзании вода добывается с помощью водокачек, и часто бывает, что водокачки требует ремонта или неожиданно ломаются, оставляя без воды целые деревни. 

Задача участников конкурса - по параметрам и статусе работоспособности некоторых водокачек предсказать возможность поломки или необходимости ремонта других водокачек.

Ссылка на контест:
https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

## Описание метода

Будем использовать GradientBoosting из пакета sklearn. Отметим, что в будущем планируется сравнить эту реализацию метода Gradient Boosting с его реализацией из пакета XGBoost. 

### Постановка задачи

Пусть есть обучающая выборка ${(x_1, y_1), ..., (x_n, y_n)}$, где $x_i$ - набор параметров, $y_i$ - метка для этого набора. Пусть есть функция потерь $L(x, F(x))$. Требуется построить модель - такую функцию $F(x)$, которая бы минимизировала бы функцию потерь.

### Краткое описание метода

Метод градинтного бустинга строит итоговую модель F(x) как взвешенную сумму так называемых _слабых моделей_ $F_m(x),\ m = 1, ..., M$. На каждом шаге к предыдущей слабой модели добавляется "невязка" $h_m(x)$ - частная производная $L(x, F_m(x))$ по $F_m(x)$ со знаком минус. Для работы алгоритма требуется, чтобы $L(x, F(x))$ была дифференцируема в достаточно большой области.

### "Наивный" алгоритм

1. Инициализация:
$$F_0(x) = argmin_\rho \sum_{i = 0}^{n}L(y_i, \rho)$$
2. На каждом шаге m, m = 1, ..., M построим слабую модель $F_m(x)$.
Представим модель $F_{m}(x)$ как сумму предыдущей модели и слагаемого $h_m(x)$, которое улучшило бы модель:

$$F_{m}(x) = F_{m - 1}(x) + h_m(x)$$

Цель: построить $F_{m}(x)$ так, чтобы $F_{m}(x)$ минимизировала функцию потерь, то есть была бы аппроксимацией $argmin_F \mathbb{E}_{x, y}L(y, F(x))$, где $\mathbb{E}$ - математическое ожидание.

Поэтому положим:
$$h_m(x) = argmin_{h\in H}\sum_{i = 0}^{n}L(y_i, F_{m - 1}(x_i) + h_m(x_i))$$
То есть:
$$F_m(x) = F_{m - 1}(x) + h_m(x) = F_{m - 1}(x) + argmin_{h\in H}\sum_{i = 0}^{n}L(y_i, F_{m - 1}(x_i) + h_m(x_i))$$
Здесь мы рассматриваем функции $h(x)$ из некоторого класса $H$.

__Проблема:__ на практике считать $argmin_{h\in H}$ - вычислительно очень сложная задача. Поэтому $h_m(x)$ вычисляют как частную производную функции потерь $L(x, F_m(x))$ по $F_m(x)$ с противоположным знаком. Такая $h_m(x)$ добавляется с весом $\rho_m$:

$$F_{m}(x) = F_{m - 1}(x) - \rho_m h_m(x)$$

$$h_m(x) = \sum_{i = 0}^{n}\frac{\partial L(y_i, F(x_i)}{\partial F(x_i)} |_{F(x) = F_{m - 1}(x)}$$

$$\rho_m = argmin_{\rho} \sum_{i = 0}^{n} L(y_i, F_{m-1}(x_i) - \rho h_m(x)$$

На практике такой подход дает очень хорошее приближение минимума в случае, когда класс функций $H$ конечен. 

3. Итоговая модель:
$$F(x) = \sum_{m = 0}^{M}\rho_i h_i(x) + const$$
 
 
### Алгоритм в общем случае

В более общем случае в качестве $h_m(x)$ используются модели, обученные на аппроксимированной выборке ${(x_1, r_{1m}), ..., (x_n, r_{nm})}$, где $r_{im}$ - метки, построенные в соответствии с моделью $F_{m - 1}(x)$. В нашем случае в качестве $h_m(x)$ использованы решающие деревья, $M$ = 500, максимальная глубина каждого дерева - 5.

1. Инициализация:
$$F_0(x) = argmin_\rho \sum_{i = 0}^{n}L(y_i, \rho)$$

2. На каждом шаге m, m = 1, ... M:
    а. Считаем _псевдо-остатки_ $r_{im}$:
    $$r_{im} = - \frac{\partial L(y_i, F(x_i)}{\partial F(x_i)} |_{F(x) = F_{m - 1}(x)},\ i = 1, ..., n$$
    б. Строим базовую модель $h_m(x)$ как модель на псевдо-остатках, то есть на выборке ${(x_1, r_{1m}), ..., (x_n, r_{nm})}$.
    в. Находим вес $\rho_m$ для найденной $h_m(x)$:
    $$\rho_m = argmin_{\rho} \sum_{i = 0}^{n} L(y_i, F_{m-1}(x_i) + \rho h(x)$$
    г. Положим:
    $$F_{m}(x) = F_{m - 1}(x) + \rho_m h_m(x)$$
    
3. Итоговая модель: $F_M(x)$.

## Загрузка данных

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier

Для работы с данными используем объект DataFrame пакета pandas.

Примем следующие обозначения:
* X - вектор списков признаков (учебных или тестовых)
* Y - вектор предсказаний (учебных или тестовых)

In [5]:
trainX = pd.read_csv('./dataT/train.csv', sep=',')
trainY = pd.read_csv('./dataT/train_labels.csv', sep=',')
testX = pd.read_csv('./dataT/test.csv', sep=',')

In [5]:
print(type(trainX))

<class 'pandas.core.frame.DataFrame'>


Посмотрим, что представляют из себя данные. Выведем первые 5 строк таблицы входных данных:

In [6]:
trainX.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


## Описание соревнования и данных

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

Суть задания - определить по набору признаков одно из трёх возможных состояний водокачки:
* исправна
* пока что исправна и требует ремонта
* неисправна

Данные относятся к водокачкам страны Танзании в Африке, где от состояния водокачек зависят судьбы местных жителей.

Полный перечень столбцов таблицы с пояснениями:

* amount_tsh - Total static head (amount water available to waterpoint)
* date_recorded - The date the row was entered
* funder - Who funded the well
* gps_height - Altitude of the well
* installer - Organization that installed the well
* longitude - GPS coordinate
* latitude - GPS coordinate
* wpt_name - Name of the waterpoint if there is one
* num_private -
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location
* region_code - Geographic location (coded)
* district_code - Geographic location (coded)
* lga - Geographic location
* ward - Geographic location
* population - Population around the well

* public_meeting - True/False
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* permit - If the waterpoint is permitted
* construction_year - Year the waterpoint was constructed
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
* management - How the waterpoint is managed
* management_group - How the waterpoint is managed
* payment - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint

# Работа с данными

## 1) Размер набора данных

Около 19 МБайт. Таблицы имеют по 40 колонок, в выходной таблице должно быть только две колонки: id водокачки и её предсказанное состояние.

In [7]:
print (trainX.shape, trainY.shape, testX.shape)

(59400, 40) (59400, 2) (14850, 40)


## 2) Список названий колонок

Аттрибут columns. Выпишем точные названия колонок, они понадобятся в дальнейшем для построения модели.

In [8]:
trainX.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

## 3) Смотрим информацию по типам столбцов

Как видно, много строковых (30), немного целочисленных (7), мало вещественных (3) типов.
Строковые можно отнести к категориальным, целочисленные и вещественные к количественным и порядковым.

In [9]:
trainX.info(verbose = None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

## 4) Характеристики каждого признака

Посчитаем характеристики целочисленных и вещественных столбцов.

Заметим: count - количество колонок - везде одинаковое число 59 400, так как в таблице заполнены все ячейки

In [10]:
trainX.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


## 5) Выводим списки регионов, фирм-установщиков и др.

С помощью метода value_counts() объединим столбец "region" в тестовой и тренировочной выборке и посчитаем количество строк каждого из регионов.

In [11]:
val = trainX['region'].value_counts()
val1 = testX['region'].value_counts()

In [12]:
val = val.append(val1)
print(val)

Iringa           5294
Shinyanga        4982
Mbeya            4639
Kilimanjaro      4379
Morogoro         4006
Arusha           3350
Kagera           3316
Mwanza           3102
Kigoma           2816
Ruvuma           2640
Pwani            2635
Tanga            2547
Dodoma           2201
Singida          2093
Mara             1969
Tabora           1959
Rukwa            1808
Mtwara           1730
Manyara          1583
Lindi            1546
Dar es Salaam     805
Shinyanga        1311
Iringa           1305
Mbeya            1119
Kilimanjaro      1115
Morogoro         1032
Kagera            858
Mwanza            795
Arusha            761
Kigoma            717
Pwani             696
Ruvuma            666
Tanga             639
Dodoma            578
Tabora            507
Mara              482
Singida           443
Rukwa             434
Mtwara            414
Manyara           389
Lindi             374
Dar es Salaam     215
Name: region, dtype: int64


Регионов малое обозримое количество и водокачки неплохо (в плане равномерности) распределены по ним. Есть даже Килиманжаро. Значит, это предположительно хороший признак, его стоит включить в модель.

Данный признак - категориальный. Заменим его со строкового типа на целочисленный прямым преобразованием: первому названию присвоим значение 0, второму - 1 и т.д.

In [13]:
#vval = val.keys()
#print(vval[3])

#print(val['Kilimanjaro'])
#print(val['Mbeya'])
for ii in range(0, len(val)):
    val[ii]=ii

In [14]:
#def f1(str):
#    return val(str)
#map(f1, trainX['region'])['Mbeya']

print(type(val))
for ii in range(0, len(val)):
    print(val.keys()[ii])
    trainX.replace(to_replace = val.keys()[ii], value = ii, inplace = True)
    testX.replace(to_replace = val.keys()[ii], value = ii, inplace = True)

<class 'pandas.core.series.Series'>
Iringa
Shinyanga
Mbeya
Kilimanjaro
Morogoro
Arusha
Kagera
Mwanza
Kigoma
Ruvuma
Pwani
Tanga
Dodoma
Singida
Mara
Tabora
Rukwa
Mtwara
Manyara
Lindi
Dar es Salaam
Shinyanga
Iringa
Mbeya
Kilimanjaro
Morogoro
Kagera
Mwanza
Arusha
Kigoma
Pwani
Ruvuma
Tanga
Dodoma
Tabora
Mara
Singida
Rukwa
Mtwara
Manyara
Lindi
Dar es Salaam


In [15]:
#trainX['region']

Регионы заменены на целочисленные индексы. Теперь рассмотрим поле имени основателя:

In [16]:
val = trainX['funder'].value_counts()
val1 = testX['funder'].value_counts()
val = val.append(val1)
val

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Kkkt                      1287
World Vision              1246
Unicef                    1057
Tasaf                      877
District Council           843
Dhv                        829
Private Individual         826
Dwsp                       811
0                          777
Norad                      765
Germany Republi            610
Tcrs                       602
Ministry Of Water          590
Water                      583
Dwe                        484
Netherlands                470
Hifab                      450
Adb                        448
Lga                        442
Amref                      425
Fini Water                 393
Oxfam                      359
Wateraid                   333
Rc Church                  321
Isf                        316
                          ... 
Nyabasamba                   1
Robert  

Видим:

* крупного застройщика водокачек под названием 0
* ещё одного крупного застройщика Private Individual - частный застройщик
* большой хвост таблицы из частных водокачек, значит, имена их funder'ов можно попробовать заменить на некого единого обобщенного частного установщика

Обе эти особенности могут плохо сказаться на точности предсказания модели. Поэтому заменим установщика 0, Private Individual, а также все установщиков с количеством водокачек меньше 50 на одного обощенного установщика, назовем его Noname.

In [17]:
#print(val['Roman'])
for ii in range(0, len(trainX)):
    if (pd.isnull(trainX['funder'][ii])):
        trainX.at[ii, 'funder'] = "Noname"
    elif (val[trainX['funder'][ii]].any() < 50):
        #print(trainX['funder'][ii])   
        #print(val[trainX['funder'][ii]])
        trainX.at[ii, 'funder'] = "Noname"

In [18]:
for ii in range(0, len(testX)):    
    if (pd.isnull(testX['funder'][ii])):
        testX.at[ii, 'funder'] = "Noname"
    else:
        #print(trainX['funder'][ii])
        if (val[testX['funder'][ii]].any() < 50):
            #print(trainX['funder'][ii])   
            #print(val[trainX['funder'][ii]])
            testX.at[ii, 'funder'] = "Noname"

In [19]:
trainX.replace(to_replace = "0", value="Noname", inplace = True)
trainX.replace(to_replace = "Private Individual", value="Noname", inplace = True)
trainX['funder'].value_counts()

testX.replace(to_replace = "0", value="Noname", inplace = True)
testX.replace(to_replace = "Private Individual", value="Noname", inplace = True)

Приведём поле "основатель" к целочисленному виду:

In [20]:
val = trainX['funder'].value_counts()
val1 = testX['funder'].value_counts()
val = val.append(val1)
for ii in range(0, len(val)):
    val[ii]=ii

for ii in range(0, len(val)):
    trainX.replace(to_replace = val.keys()[ii], value = ii, inplace = True)
    testX.replace(to_replace = val.keys()[ii], value = ii, inplace = True)
#trainX['funder']

Проверим на совпадения колонки source и source_type. А ещё присутствует колонка source_class

In [21]:
val1 = trainX['source']
val2 = trainX['source_type']
count = 0
for ii in range(0,len(val1)):
    if (val1[ii] == val2[ii]):
        count += 1
        #print ("misfit")
print ("совпадение в ", int(count/len(val1)*100.0), "% случаев")


совпадение в  62 % случаев


То же самое относительно payment и payment_type

In [22]:
val1 = trainX['payment']
val2 = trainX['payment_type']
count = 0
for ii in range(0,len(val1)):
    if (val1[ii] == val2[ii]):
        count += 1
        #print ("misfit")
print ("совпадение в ", int(count/len(val1)*100.0), "% случаев")

совпадение в  58 % случаев


Обнаружена чрезмерная схожесть значений столбцов. Не будем использовать дублирующиеся столбцы.

## 6) Определим средний возраст водокачек

In [23]:
trainX['construction_year'].mean()

1300.6524747474748

Средний год постройки = 1300 . . . Выглядит подозрительно.

Подсчитаем процент записей, в которых "год постройки" заполнен нулём:

In [24]:
print(int(len(trainX[trainX['construction_year'] == 0])/len(trainX)*100.0), "%")

34 %


Возраст, субъективно, один из ключевых параметров, влияющих на отказ водокачки при прочих фиксированных параметрах. И с этим придётся что-то делать. Во всех строках с нулевым годом постройки заменим нули на срединий год без учета нулевых строк:

In [25]:
avg = 0
count = 0
for ii in range(0, len(trainX)):
    if (trainX['construction_year'][ii] != 0):
        avg += trainX['construction_year'][ii]
        count += 1
avg /= count
print("средний возраст водокачек с заполненным полем =", avg)
trainX['construction_year'].replace(to_replace = 0, value = avg, inplace = True)
testX['construction_year'].replace(to_replace = 0, value = avg, inplace = True)
print("среднйи возраст после заполнения пробелов", trainX['construction_year'].mean())

средний возраст водокачек с заполненным полем = 1996.81468559
среднйи возраст после заполнения пробелов 1996.81468559


## 7) Корреляции

Построим таблицу взаимных корреляций числовых признаков. 

Из таблицы видно, например, что признаки region_code и district_code сильно коррелирует, значит, хотя бы один из этих признаков стоит исключить из модели.

Мы пробовали строить график взаимных корреляций в виде тепловой карты, но такой график получился малоинформативным.

In [26]:
trainX.corr()

,id,amount_tsh,funder,gps_height,longitude,latitude,num_private,region,region_code,district_code,population,construction_year
id,1.000000,-0.005321,NaN,-0.004692,-0.001348,0.001718,-0.002629,0.003246,-0.003028,-0.003044,-0.002813,-0.003356
amount_tsh,-0.005321,1.000000,NaN,0.076650,0.022134,-0.052670,0.002944,-0.020419,-0.026813,-0.023599,0.016288,0.006036
funder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gps_height,-0.004692,0.076650,NaN,1.000000,0.149155,-0.035751,0.007237,-0.048600,-0.183521,-0.171233,0.135003,-0.004493
longitude,-0.001348,0.022134,NaN,0.149155,1.000000,-0.425802,0.023873,0.188238,0.034197,0.151398,0.086590,0.008830
latitude,0.001718,-0.052670,NaN,-0.035751,-0.425802,1.000000,0.006837,-0.061388,-0.221018,-0.201020,-0.022152,0.029159
num_private,-0.002629,0.002944,NaN,0.007237,0.023873,0.006837,1.000000,0.009891,-0.020377,-0.004478,0.003818,-0.003233
region,0.003246,-0.020419,NaN,-0.048600,0.188238,-0.061388,0.009891,1.000000,0.392629,0.198037,0.174214,-0.035931
region_code,-0.003028,-0.026813,NaN,-0.183521,0.034197,-0.221018,-0.020377,0.392629,1.000000,0.678602,0.094088,-0.009402
district_code,-0.003044,-0.023599,NaN,-0.171233,0.151398,-0.201020,-0.004478,0.198037,0.678602,1.000000,0.061831,0.006739


# Строим модель

Создаём объект модели:

In [28]:
model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=5,
        #subsample=0.5,
        max_features="auto"
        # random_state=seed
    )

Делаем предвыборку нужных столбцов и приводим типы данных

In [29]:
ctrainX = trainX[["construction_year", "population", "gps_height", "funder", "region"]] #, "payment", "source"]]
ctrainX = ctrainX.as_matrix()
ctrainX = ctrainX.astype(float)

trainY.replace(to_replace="functional", value=0, inplace=True)
trainY.replace(to_replace="non functional", value=1, inplace=True)
trainY.replace(to_replace="functional needs repair", value=2, inplace=True)
ctrainY = trainY.as_matrix()
ctrainY = ctrainY[:, 1]
ctrainY = ctrainY.astype(float)

a=0
b=0
c=0
#for ii in range(0, len(trainY)):
#    if (trainY[ii] == 1):
#        a += 1
#    if (trainY[ii] == 2):
#        b += 1
#    if (trainY[ii] == 3):
#        c += 1
        
print(a,b,c)    

print(len(ctrainX), len(ctrainY))

ctestX = testX[["construction_year", "population", "gps_height", "funder", "region"]]
ctestX = ctestX.as_matrix()
print(ctestX)
ctestX = ctestX.astype(float)

0 0 0
59400 59400
[[ 2012.   321.  1996.     0.    18.]
 [ 2000.   300.  1569.     0.     5.]
 [ 2010.   500.  1567.     0.    13.]
 ..., 
 [ 2010.   200.  1476.     0.    13.]
 [ 2009.   150.   998.     0.     9.]
 [ 2008.    40.   481.     0.     9.]]


Обучаем модель и делаем предсказание

In [30]:
model.fit(ctrainX, ctrainY)

#predictions = model.predict_proba(testX)[:,1]
predictions = model.predict_proba(ctestX)

В качетсве результата получаем таблицу вероятностей каждого из трёх состояний:

In [31]:
predictions

array([[ 0.80687034,  0.17869014,  0.01443952],
       [ 0.78140004,  0.16928775,  0.0493122 ],
       [ 0.79554413,  0.16082399,  0.04363189],
       ..., 
       [ 0.84689904,  0.12725302,  0.02584793],
       [ 0.72326273,  0.24022143,  0.03651584],
       [ 0.69222586,  0.289074  ,  0.01870014]])

In [32]:
len(predictions)

14850

In [33]:
answ = predictions[:, 0].astype(int)
for ii in range (0, len(predictions)):
    answ[ii] = predictions[ii].argmax();

#answdf = pd.DataFrame(data=answ)
#answdf.to_csv('./dataT/test_labels.csv', sep=',')

#pred = [list(x) for x in predictions]
#[my_predict.index(max(my_predict)) + 1 for my_predict in pred]

Выбрали максимально вероятное состояние, конвертируем в выходной формат и записываем в файл:

In [34]:
new_data = testX[['id']].copy()

In [35]:
testY = pd.read_csv('./dataT/SubmissionFormat.csv', sep=',')
for ii in range(0, len(testY)):
    testY.at[ii, 'status_group'] = answ[ii]
    
answdf = pd.DataFrame(data=testY)
answdf.to_csv('./dataT/output.csv', sep=',')

Посмотрим на количество разных предсказаний:

In [36]:
answ = predictions[:, 0].astype(int)

a = 0
b = 0
c = 0
for ii in range (0, len(predictions)):
    answ[ii] = predictions[ii].argmax() + 1;
    if (answ[ii] == 1):
        a += 1
    if (answ[ii] == 2):
        b += 1
    if (answ[ii] == 3):
        c += 1

print(a,b,c)

11153 3501 196


Получается, в прогнозе больше всего (11 153) работающих водокачек, 3501 неработающих и 196 требуют ремонта.

## Результаты

Итак, мы проанализировали данные, провели обработку данных с целью оптимизации качества и скорости построения модели, построили модель классификатора с помощью пакета sklearn и сделали предсказание с помощью построенной модели.

Система оценивает участников по отправляемым ими предсказания для тестовой выборки, правильные ответы для которой доступны только организаторам. Предсказание, сделанное данным алгоритмом, содержит 64% правильных предсказаний, что лучше чем у 75% участников. Всего в соревновании участвуют около 4000 человек.


Возможные направление развития:
* проверить, действительно ли убранные столбцы негативно влияют на точность предсказаний
* попробовать принципиально другие модели

Данный проект был успешно защищен в рамках итогового проекта курса "Введение в машнинное обучение" Техносферы.